# TOPIC MODELLING[LDA:LatentDirichletAllocation]-BANK CONSUMER COMPLAINS

In [1]:
#INSTALLING AND IMPORTING ALL THE LIBRARIES REQUIRED FOR PRE-PROCESSING

#pip install autocorrect
#nltk.download('punkt')
#pip install unidecode
#pip install autocorrect
#pip install num2words
#nltk.download('wordnet')
#pip install --upgrade gensim
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
#!pip install emot
#from emot.emo_unicode import UNICODE_EMO, EMOTICONS
#from emoticon_list import UNICODE_EMO
#from emoticons_list import EMOTICONS
#from emoticons_list import EMOTICONS_EMO

from nltk.corpus import stopwords
import spacy
import gensim
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
from nltk import word_tokenize
import nltk
import unidecode
from string import punctuation
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
%matplotlib inline
from nltk.stem import PorterStemmer
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random                              # pseudo-random number generator
nltk.download('stopwords')                 # download the stopwords from NLTK
import re                                  # library for regular expression operations
import string                              # for string operations
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
import pandas as pd                        # Python Pandas Library
import numpy as np                         # Python Numpy Library

E:\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sarth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sarth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
complaints = pd.read_csv("consumer_compliants.csv", sep=',',quotechar='"')

In [4]:
complaints.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt was incurred due to medical malpract...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,None,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704


In [5]:
complaints["Consumer complaint narrative"][4]

'This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never stopped it from being reported when I specifically told them not to, due to the circumstances above.'

In [6]:
complaints['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [7]:
complaints['Company'].value_counts()

CITIBANK, N.A.                           3226
CAPITAL ONE FINANCIAL CORPORATION        2711
BANK OF AMERICA, NATIONAL ASSOCIATION    2580
JPMORGAN CHASE & CO.                     2409
WELLS FARGO & COMPANY                    2001
                                         ... 
Happy Money, Inc.                           1
HOMESTAR FINANCIAL CORPORATION              1
Emerald AR Systems, LLC                     1
Fusion Auto Finance                         1
KAHRS LAW OFFICES, PA                       1
Name: Company, Length: 2197, dtype: int64

In [8]:
#Dropping Columns which are not necessary for our task (Topic Modelling)
complaints_df=complaints[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complains'})

In [9]:
complaints_df.head()

,complains,Product,Company
0,This auto loan was opened on XX/XX/2020 in XXX...,Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,In XXXX of 2019 I noticed a debt for {$620.00}...,Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX an...",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in...",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,This debt was incurred due to medical malpract...,Debt collection,"Merchants and Professional Bureau, Inc."


In [10]:
complaints_df.shape

(57453, 3)

In [11]:
complaints_df.dtypes

complains    object
Product      object
Company      object
dtype: object

In [12]:
#To display the entire text which is by default wrapped up by the pandas as seen above when we are not setting it up
pd.set_option('display.max_colwidth', None)
complaints_df

,complains,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never 

## NLP Text Pre-processing

In [13]:
from bs4 import BeautifulSoup

In [14]:
#Function for PreProcessing Complains
def preprocess_complains(text):
    
    
    # conversion of all the tweets in lower Case format
    lowercase_conversion = text.lower()
    
    #remove html tags
    parser = BeautifulSoup(lowercase_conversion, "html.parser")
    html_removal = parser.get_text(separator = " ")

    
    # removal of hyperlinks/URL
    url_pattern = r'https?://\S+|www\.\S+'
    url_removal = re.sub(pattern=url_pattern, repl=' ', string=html_removal)
    
    #removing \n
    n_removal = re.sub(r'\n', '', url_removal)
  
    return n_removal

In [15]:
#Complain Pre-Processing
records=[]
results = complaints_df['complains']
for item in results:
    complain_preprocessing = preprocess_complains(item)
    if complain_preprocessing:
        records.append(complain_preprocessing)
complaints_df['complains_PreProcessed']=records

In [16]:
complaints_df.head()

,complains,Product,Company,complains_PreProcessed
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION,"this auto loan was opened on xx/xx/2020 in xxxx, nc with bb & t in my name. i have never been to north carolina and i have never been a resident. i have filed a dispute twice through my credit bureaus but both times bb & t has claimed that this is an accurate loan. which i wasn't aware of until today. i have tried to contact bb & t multiple times but i have never gotten through to a live person. i do n't drive and i have never owned a car before. i didn't have any knowledge of this account until i checked xxxxxxxx xxxx and noticed it. i've tried twice to dispute it. additionally i never received any bills or information about this account. this is my last resort in trying to remove this fraudulent loan off of my account."
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings,"in xxxx of 2019 i noticed a debt for {$620.00} on my credit which i believed was mine i thought speedy cash had bought one of my old debts and sold it to xxxx xxxx xxxx xxxx. i contacted xxxx xxxx xxxx xxxx and after several attempts of giving my full name, nothing came up in their system. i gave my social and the rep said the account popped up but did not tell me that the account was under someone elses name and continued to let me make a payment. the payment was for {$120.00}. confirmation number-xxxx. after realizing it was not my account, i called back to get my money back and inform them of the mistake. i was told i needed to mail them an ftc report and dispute letter to get my money back. i completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and i would need to contact them. after contacting them i was again told that i can not get my money back. the issue im having is this representative at xxxx xxxx played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back."
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However

### TOKENIZATION, LEMMATIZATION, STOPWORD REMOVAL & PUNCTATION REMOVAL

In [18]:
lemma = WordNetLemmatizer()
stopwords_english = stopwords.words('english')
print('Stop words\n')
print(stopwords_english)
print('\nPunctuation\n')
print(string.punctuation)

Stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so

In [19]:
#adding following punction in the string ”“
punc=list(string.punctuation)
all_punctuation = ['”','“']
all_punctuation.extend(punc)
punc_str = ""
new_punctuations=punc_str.join(all_punctuation)
new_punctuations

'”“!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [28]:
stemmer = PorterStemmer()

In [83]:
def final_preprocess(text):
    
    #Tokenization & Further Pre-Processing
    tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 2 and len(word.strip('xx/')) > 2 and len(re.sub('\d+','',word.strip('Xx/')))>3) ] 
    
    #Lemmatization of the tokenized words
    lem = [lemma.lemmatize(item) for item in tokens]
    #stems = [stemmer.stem(item) for item in tokens]   # Stemming can also be used but as we are finding out the Topic from the complaints we want meaningful words.
    
    #Stopwords and Punctutation Removal
    complains_without_sw_punctuations = []
    for word in lem:                                   # Go through every word in your tokens list
        if (word not in stopwords_english and          # remove stopwords
            word not in new_punctuations):             # remove punctuation
            complains_without_sw_punctuations.append(word)
            
    #the following code can be futher reduced by list comprehension over lemmatixation function
    #lem = [lemma.lemmatize(item) for item in tokens if (item not in stopwords_english and item not in new_punctuations)]
            
    final_preprocess =' '.join(complains_without_sw_punctuations)
    return final_preprocess

In [84]:
# Final PreProcessing
records=[]
results = complaints_df['complains_PreProcessed']
for item in results:
    final_preprocessing = final_preprocess(item)
    if final_preprocessing:
        records.append(final_preprocessing)
    else:
        records.append("NA")
complaints_df['complains_Final_Processed']=records

In [85]:
complaints_df.head()

,complains,Product,Company,complains_PreProcessed,complains_Final_Processed
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION,"this auto loan was opened on xx/xx/2020 in xxxx, nc with bb & t in my name. i have never been to north carolina and i have never been a resident. i have filed a dispute twice through my credit bureaus but both times bb & t has claimed that this is an accurate loan. which i wasn't aware of until today. i have tried to contact bb & t multiple times but i have never gotten through to a live person. i do n't drive and i have never owned a car before. i didn't have any knowledge of this account until i checked xxxxxxxx xxxx and noticed it. i've tried twice to dispute it. additionally i never received any bills or information about this account. this is my last resort in trying to remove this fraudulent loan off of my account.",auto loan opened name never north carolina never resident filed dispute twice credit bureau time claimed accurate loan aware today tried contact multiple time never gotten live person drive never owned knowledge account checked noticed tried twice dispute additionally never received bill information account last resort trying remove fraudulent loan account
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings,"in xxxx of 2019 i noticed a debt for {$620.00} on my credit which i believed was mine i thought speedy cash had bought one of my old debts and sold it to xxxx xxxx xxxx xxxx. i contacted xxxx xxxx xxxx xxxx and after several attempts of giving my full name, nothing came up in their system. i gave my social and the rep said the account popped up but did not tell me that the account was under someone elses name and continued to let me make a payment. the payment was for {$120.00}. confirmation number-xxxx. after realizing it was not my account, i called back to get my money back and inform them of the mistake. i was told i needed to mail them an ftc report and dispute letter to get my money back. i completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and i would need to contact them. after contacting them i w

In [90]:
#The following Record has a irrelevant data that is NA so we will drop it
complaints_df[complaints_df['complains_Final_Processed']=="NA"]

,complains,Product,Company,complains_PreProcessed,complains_Final_Processed
54015,XXXX XXXX &,Checking or savings account,"BANK OF AMERICA, NATIONAL ASSOCIATION",xxxx xxxx &,NA


In [91]:
complaints_df=complaints_df.drop(index = 54015)

In [92]:
complaints_df[complaints_df['complains_Final_Processed']=="NA"]

,complains,Product,Company,complains_PreProcessed,complains_Final_Processed


In [93]:
#Resetting the index
complaints_df=complaints_df.reset_index(drop=True)

In [96]:
complaints_df[54014:54016] #Reset Successful

complains  \
54014                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

## UNSUPERVISED MACHINE LEARNING

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import decomposition

In [98]:
X_train, X_hold = train_test_split(complaints_df, test_size=0.3, random_state=111)

In [99]:
X_train.shape

(40216, 5)

In [100]:
X_train.head()

,complains,Product,Company,complains_PreProcessed,complains_Final_Processed
44932,"I have used a Capital One for Business card for many years, always paying off the monthly balance, averaging {$10000.00}, on time. \nIn the past 2 years there have been multiple instances of fraudulent activity, often caught while charges were still pending, even tho my card was never out of my possession. Capital One has always replaced my card. \nThe most recent replacement card was sent to me early XXXX. I activated it. When I needed to use it for a purchase XX/XX/XXXX it was declined. I logged in and couldnt access my account information. Weird. \nI called Capital One and was informed my account had been charged off. \nWhy? And why wasnt I notified? Eventually I was told it was because I was deemed high risk because fraud had occurred 5 times. OK. I can accept being dropped for being the victim. But I was never notified. Shutting down my card also left me without cell phone activation. And to take away my online access to my account also made it so I couldnt see my statement, pay my bill, and use my reward points. \nCapital One cut me off without notification. This put me at risk. I feel it is an outrageous way to treat a solid customer. I was informed in XXXX my SS # and other personal information was stolen in a security breach at Capital One. Great.",Credit card or prepaid card,CAPITAL ONE FINANCIAL CORPORATION,"i have used a capital one for business card for many years, always paying off the monthly balance, averaging {$10000.00}, on time. in the past 2 years there have been multiple instances of fraudulent activity, often caught while charges were still pending, even tho my card was never out of my possession. capital one has always replaced my card. the most recent replacement card was sent to me early xxxx. i activated it. when i needed to use it for a purchase xx/xx/xxxx it was declined. i logged in and couldnt access my account information. weird. i called capital one and was informed my account had been charged off. why? and why wasnt i notified? eventually i was told it was because i was deemed high risk because fraud had occurred 5 times. ok. i can accept being dropped for being the victim. but i was never notified. shutting down my card also left me without cell phone activation. and to take away my online access to my account also made it so i couldnt see my statement, pay my bill, and use my reward points. capital one cut me off without notification. this put me at risk. i feel it is an outrageous way to treat a solid customer. i was informed in xxxx my ss # and other personal information was stolen in a security breach at capital one. great.",used capital business card many year always paying monthly balance averaging time past year multiple instance fraudulent activity often caught charge still pending even card never possession capital always replaced card recent replacement card sent early activated needed purchase declined logged couldnt access account information weird called capital informed account charged wasnt notified eventually told deemed high risk fraud occurred time accept dropped victim never notified shutting card also left without cell phone activation take away online access account also made couldnt statement bill reward point capital without notification risk feel outrageous treat solid customer informed personal information stolen security breach capital great
2078,I have disputed this account and have reached out to the company/creditors MANY times but failed. This account is being reported inaccurately and incorrectly on my credit report.,Debt collection,Resurgent Capital Services L.P.,i have disputed this account and have reached out to the company/creditors many times but failed. this account is being reported inaccurately and incorrectly on my credit report.,disputed account reached company/creditors many time failed account reported inaccurately incorrectly credit report
35921,"i had 2 Kohl 's Charg

In [101]:
X_train.dtypes

complains                    object
Product                      object
Company                      object
complains_PreProcessed       object
complains_Final_Processed    object
dtype: object

In [102]:
type(complaints_df['complains_Final_Processed'])

pandas.core.series.Series

In [103]:
type(complaints_df['complains'])

pandas.core.series.Series

In [104]:
type(complaints_df['complains_PreProcessed'])

pandas.core.series.Series

In [109]:
tfidfvector=TfidfVectorizer(max_df=0.75,min_df=50, max_features=10000,lowercase =False, use_idf=False, norm=None )
tf_vectors = tfidfvector.fit_transform(X_train["complains_Final_Processed"])

In [106]:
tf_vectors.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [108]:
tfidfvector.get_feature_names()

['10',
 '11',
 '120',
 '19',
 '2019',
 '2020',
 '2295',
 '30',
 '60',
 'aadvantage',
 'abide',
 'ability',
 'able',
 'above',
 'abruptly',
 'absent',
 'absolute',
 'absolutely',
 'absurd',
 'abuse',
 'abused',
 'abusive',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'access',
 'accessed',
 'accessible',
 'accessing',
 'accident',
 'accidentally',
 'accord',
 'accordance',
 'according',
 'accordingly',
 'account',
 'accountability',
 'accountable',
 'accountant',
 'accounting',
 'accrue',
 'accrued',
 'accruing',
 'acct',
 'accumulated',
 'accuracy',
 'accurate',
 'accurately',
 'accusation',
 'accused',
 'acknowledge',
 'acknowledged',
 'acknowledgement',
 'acknowledging',
 'acknowledgment',
 'acquiescence',
 'acquire',
 'acquired',
 'acquiring',
 'acquisition',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'activate',
 'activated',
 'activation',
 'active',
 'actively',
 'activity',
 'actual',
 'actually',
 'add',
 'added',
 'adding',
 'addition',
 'addit

In [162]:
lda_processing = decomposition.LatentDirichletAllocation(n_components=6, max_iter=3, learning_method='online', learning_offset=50, n_jobs=-1, random_state=111)

toutput_vector = lda_processing.fit_transform(tf_vectors)
topic_component = lda_processing.components_

In [163]:
toutput_vector

array([[0.70229194, 0.00166478, 0.22946125, 0.0632629 , 0.00166226,
        0.00165688],
       [0.01119704, 0.01115564, 0.0112106 , 0.0112102 , 0.94407764,
        0.01114889],
       [0.06267116, 0.00262147, 0.10086496, 0.8286201 , 0.00261382,
        0.0026085 ],
       ...,
       [0.58564888, 0.00270522, 0.40350699, 0.00271503, 0.00272031,
        0.00270356],
       [0.92982383, 0.01390381, 0.01399952, 0.01408328, 0.01426978,
        0.01391977],
       [0.00123997, 0.00124613, 0.00124156, 0.43498045, 0.16794744,
        0.39334445]])

In [164]:
num_words=15

vocab = np.array(tfidfvector.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in topic_component])
topics = [' '.join(t) for t in topic_words]

In [165]:
topics

['account card bank credit charge check chase transaction fraud dispute closed received money would customer',
 'complaint bank offer well state financial term consumer case bonus time would court fargo business',
 'call told would called time said back phone company asked could number never received money',
 'payment month paid account interest late amount balance made mortgage statement would insurance time credit',
 'debt credit account report collection company information letter reporting agency sent bureau please provide creditor',
 'loan mortgage home document modification application student closing request property lender foreclosure process income servicing']

In [166]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.complains))]
complain_df_document_topic = pd.DataFrame(np.round(toutput_vector, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(complain_df_document_topic.values, axis=1)
complain_df_document_topic['dominant_topic'] = significant_topic

In [167]:
complain_df_document_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.70,0.00,0.23,0.06,0.00,0.00,0
Doc1,0.01,0.01,0.01,0.01,0.94,0.01,4
Doc2,0.06,0.00,0.10,0.83,0.00,0.00,3
Doc3,0.69,0.28,0.01,0.01,0.01,0.01,0
Doc4,0.00,0.03,0.17,0.71,0.00,0.09,3
...,...,...,...,...,...,...,...
Doc40211,0.39,0.36,0.24,0.00,0.00,0.00,0
Doc40212,0.50,0.00,0.00,0.29,0.19,0.00,0
Doc40213,0.59,0.00,0.40,0.00,0.00,0.00,0
Doc40214,0.93,0.01,0.01,0.01,0.01,0.01,0


In [168]:
complain_df_document_topic['dominant_topic'].value_counts()

0    9875
4    9739
2    9720
3    6344
5    2514
1    2024
Name: dominant_topic, dtype: int64

In [169]:
X_train.head()

,complains,Product,Company,complains_PreProcessed,complains_Final_Processed
44932,"I have used a Capital One for Business card for many years, always paying off the monthly balance, averaging {$10000.00}, on time. \nIn the past 2 years there have been multiple instances of fraudulent activity, often caught while charges were still pending, even tho my card was never out of my possession. Capital One has always replaced my card. \nThe most recent replacement card was sent to me early XXXX. I activated it. When I needed to use it for a purchase XX/XX/XXXX it was declined. I logged in and couldnt access my account information. Weird. \nI called Capital One and was informed my account had been charged off. \nWhy? And why wasnt I notified? Eventually I was told it was because I was deemed high risk because fraud had occurred 5 times. OK. I can accept being dropped for being the victim. But I was never notified. Shutting down my card also left me without cell phone activation. And to take away my online access to my account also made it so I couldnt see my statement, pay my bill, and use my reward points. \nCapital One cut me off without notification. This put me at risk. I feel it is an outrageous way to treat a solid customer. I was informed in XXXX my SS # and other personal information was stolen in a security breach at Capital One. Great.",Credit card or prepaid card,CAPITAL ONE FINANCIAL CORPORATION,"i have used a capital one for business card for many years, always paying off the monthly balance, averaging {$10000.00}, on time. in the past 2 years there have been multiple instances of fraudulent activity, often caught while charges were still pending, even tho my card was never out of my possession. capital one has always replaced my card. the most recent replacement card was sent to me early xxxx. i activated it. when i needed to use it for a purchase xx/xx/xxxx it was declined. i logged in and couldnt access my account information. weird. i called capital one and was informed my account had been charged off. why? and why wasnt i notified? eventually i was told it was because i was deemed high risk because fraud had occurred 5 times. ok. i can accept being dropped for being the victim. but i was never notified. shutting down my card also left me without cell phone activation. and to take away my online access to my account also made it so i couldnt see my statement, pay my bill, and use my reward points. capital one cut me off without notification. this put me at risk. i feel it is an outrageous way to treat a solid customer. i was informed in xxxx my ss # and other personal information was stolen in a security breach at capital one. great.",used capital business card many year always paying monthly balance averaging time past year multiple instance fraudulent activity often caught charge still pending even card never possession capital always replaced card recent replacement card sent early activated needed purchase declined logged couldnt access account information weird called capital informed account charged wasnt notified eventually told deemed high risk fraud occurred time accept dropped victim never notified shutting card also left without cell phone activation take away online access account also made couldnt statement bill reward point capital without notification risk feel outrageous treat solid customer informed personal information stolen security breach capital great
2078,I have disputed this account and have reached out to the company/creditors MANY times but failed. This account is being reported inaccurately and incorrectly on my credit report.,Debt collection,Resurgent Capital Services L.P.,i have disputed this account and have reached out to the company/creditors many times but failed. this account is being reported inaccurately and incorrectly on my credit report.,disputed account reached company/creditors many time failed account reported inaccurately incorrectly credit report
35921,"i had 2 Kohl 's Charg

In [170]:
toutput_vector_hold = lda.transform(tfidfvector.transform(X_hold["complains_Final_Processed"]))

In [172]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold.complains))]
complain_df_document_topic_hold = pd.DataFrame(np.round(toutput_vector_hold, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(complain_df_document_topic_hold.values, axis=1)
complain_df_document_topic_hold['dominant_topic'] = significant_topic

In [173]:
complain_df_document_topic_hold

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.62,0.08,0.27,0.01,0.01,0.01,0
Doc1,0.01,0.01,0.78,0.05,0.15,0.01,2
Doc2,0.01,0.01,0.01,0.01,0.97,0.01,4
Doc3,0.52,0.00,0.31,0.06,0.10,0.00,0
Doc4,0.02,0.02,0.02,0.30,0.61,0.02,4
...,...,...,...,...,...,...,...
Doc17231,0.73,0.00,0.10,0.16,0.00,0.00,0
Doc17232,0.23,0.13,0.32,0.21,0.00,0.10,2
Doc17233,0.56,0.00,0.22,0.00,0.21,0.00,0
Doc17234,0.04,0.00,0.22,0.74,0.00,0.00,3


In [176]:
complain_df_document_topic_hold['dominant_topic'].value_counts()

0    4250
2    4206
4    4160
3    2671
5    1051
1     898
Name: dominant_topic, dtype: int64

In [177]:
X_hold.head()

,complains,Product,Company,complains_PreProcessed,complains_Final_Processed
11451,"in XX/XX/XXXX i cancelled a transaction for {XXXX} and i was refunded my money. \n\nOn XX/XX/XXXX, Paypal charged me again for this {$XXXX}. I send them a letter to the address below and they have not replied. : PayPal, Inc XXXX XXXX XXXX XXXX XXXX XXXX, CA XXXX www.paypal.com XXXX I did not authorize them to charge me again. i am XXXX with medical conditions. I have not left the house in 3 months.",Credit card or prepaid card,SYNCHRONY FINANCIAL,"in xx/xx/xxxx i cancelled a transaction for {xxxx} and i was refunded my money. on xx/xx/xxxx, paypal charged me again for this {$xxxx}. i send them a letter to the address below and they have not replied. : paypal, inc xxxx xxxx xxxx xxxx xxxx xxxx, ca xxxx xxxx i did not authorize them to charge me again. i am xxxx with medical conditions. i have not left the house in 3 months.",cancelled transaction refunded money paypal charged send letter address replied paypal authorize charge medical condition left house month
24451,They constantly call and when I answer the phone theyll hang up. Im sick of the harassment and if they call me Id like for them to actually say something instead of calling up to 5 times a day. I have blocked the numbers they call on then theyll use a new number. \n\nThe company is ERC. I *think* the debt is with XXXX XXXX who opened accounts for me without my permission resulting in hundreds of dollars worth of fees Im never going to pay.,Debt collection,ERC,they constantly call and when i answer the phone theyll hang up. im sick of the harassment and if they call me id like for them to actually say something instead of calling up to 5 times a day. i have blocked the numbers they call on then theyll use a new number. the company is erc. i *think* the debt is with xxxx xxxx who opened accounts for me without my permission resulting in hundreds of dollars worth of fees im never going to pay.,constantly call answer phone theyll hang sick harassment call like actually something instead calling time blocked number call theyll number company *think* debt opened account without permission resulting hundred dollar worth fee never going
4848,On my credit report you listed that I have a collection/chargeoff from your company. Please provide signed documents with my signature or any other documents showing I supposedly have a collection/chargeoff with your company. Please provide me with all written documentation within 10 days or this is a violation and you must DELETE it from my credit report. I only want you to respond via CFPB. Thank you,Debt collection,PORTFOLIO RECOVERY ASSOCIATES INC,on my credit report you listed that i have a collection/chargeoff from your company. please provide signed documents with my signature or any other documents showing i supposedly have a collection/chargeoff with your company. please provide me with all written documentation within 10 days or this is a violation and you must delete it from my credit report. i only want you to respond via cfpb. thank you,credit report listed collection/chargeoff company please provide signed document signature document showing supposedly collection/chargeoff company please provide written documentation within day violation must delete credit report want respond cfpb thank
37440,"I was using a Citibank or Citicards credit card, Citi DiamXXXX Preferred Card for my New York XXXX XXXX membership ( the club has been closed due to the Carona Virus lockdown since XX/XX/2020 ) and the monthly fee for my passport membership was {$54.00}. New York XXXX continued to charge in spite of their clubs closing and I called Citicards repeatedly to dispute the charge. The first time I called was XX/XX/2020. I was told by Citicards that they would look into the dispute and credit my account for the amount of {$54.00}. That never happened. Instead for the charge ( which I had disputed ) due XX/XX/2020 they added a late fee and began calling me le